# 《Python进行神经网络编程》

## 导入必要的库函数

In [1]:
import numpy as np
from scipy.special import expit as sigmoid      #sigmoid激活函数
from scipy.ndimage import rotate                #旋转图像的函数

## 定义各种常数

In [2]:
InputNodes = 784                    #输入层网络节点，输入28 * 28图片，784像素点，所以输入784个节点
HiddenNodes = 200                   #隐藏层节点200哥
OutputNodes = 10                    #输出层节点，共0-9，10个数字
LearnRate = 0.1                     #学习率0.1

Epoches = 10                        #学习10个世代

## 定义神经网络对象

In [3]:
class NeutralNetwork:
    '神经网络对象'
    def __init__(self, input_nodes = 3, hidden_nodes = 3, output_nodes = 3, learn_rate = 0.3):
        '初始化神经网络'
        # 参数设置
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learn_rate
        self.af = sigmoid
        # 生成均值为0，标准差为输入节点数开平方倒数的随机权重矩阵
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
    def Train(self, input_lists, output_lists):
        '神经网络训练函数'
        # 将输入的列表转换为输入矩阵与目标矩阵
        inputs = np.array(input_lists, ndmin = 2).T
        targets = np.array(output_lists, ndmin = 2).T
        # 隐层输入等于权重乘输入层的输出
        hidden_inputs = np.dot(self.wih, inputs)
        # 隐层输出为输入施加激活函数
        hidden_outputs = self.af(hidden_inputs)

        # 输出层输入等于权重乘隐层输出
        final_inputs = np.dot(self.who, hidden_outputs)
        # 输出层输出为输入施加激活函数
        final_outputs = self.af(final_inputs)

        # 输出误差为目标值减输出值
        final_errors = targets - final_outputs
        # 反向传播误差，隐藏层误差为输出层误差乘以网络权重
        hidden_errors = np.dot(self.who.T, final_errors)
        
        # 根据反向传播的误差调整权重
        self.who += self.lr * np.dot(final_errors * final_outputs * (1.0 - final_outputs) , hidden_outputs.T)
        self.wih += self.lr * np.dot(hidden_errors * hidden_outputs * (1.0 - hidden_outputs), inputs.T)

    def Query(self, inputs_list):
        '根据已训练好的网络查询输入得到的输出'
        inputs = np.array(inputs_list, ndmin = 2).T
        # 计算隐层输出
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.af(hidden_inputs)
        # 计算输出层输出
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.af(final_inputs)

        return final_outputs

## 训练网络

In [4]:
# 打开文件获取数据
train_data_file = open('mnist_train.csv','r')
# 按行读取，获取一个列表，如果训练数据较多的话可以用readline函数一行一行地读，读一行处理一行
train_data_lists = train_data_file.readlines()
# 关闭文件
train_data_file.close()

#定义一个神经网络对象
n = NeutralNetwork(InputNodes, HiddenNodes, OutputNodes, LearnRate)
#循环Epoches个世代，提高准确率
for x in range(Epoches):
    # 输出训练的世代数，以便时刻了解训练进度
    print('Epoch: ',x+1)
    # 按行读取数据进行训练，读取到的数据是一个字符串
    for record in train_data_lists:
        #按照逗号，分开各个值，返回一个数字组成的列表
        all_values = record.split(',')
        # 调整输入值到范围（0.01，1），asfarry输入为一个字符串
        inputs = np.asfarray(all_values[1:]) /255 * 0.99 + 0.01
        # 调整输出，标准答案为1，其他为0.01
        targets = np.zeros(OutputNodes) + 0.01
        targets[int(all_values[0])] += 0.99
        # 调用神经网络训练函数训练
        n.Train(inputs, targets)
        #调整图片角度+-10度再次进行训练
        inputs_plug10_img = rotate(inputs.reshape(28,28), 10, cval = 0.01, reshape = False)
        n.Train(inputs_plug10_img.reshape(1, 784), targets)
        inputs_minus10_img = rotate(inputs.reshape(28,28), -10, cval = 0.01, reshape = False)
        n.Train(inputs_minus10_img.reshape(1, 784), targets)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10


## 测试网络

In [5]:
#从文件获取测试集数据
test_data_file = open('mnist_test.csv','r')
test_data_lists = test_data_file.readlines()
test_data_file.close()

Score = 0  #正确个数

for test_data in test_data_lists:
    # 处理输入数据调整为0.01-1的矩阵
    all_values = test_data.split(',')
    inputs = np.asfarray(all_values[1:]) /255 * 0.99 + 0.01
    # 查询经过神经网络以后的输出值
    answer = n.Query(inputs)
    # 获取输出矩阵的最大值角标，判断是否正确，如果正确score+1
    if np.argmax(answer) == int(all_values[0]):
        Score += 1

print('Accuracy Rate:  ', 100.0 * Score /len(test_data_lists), '%' )

Accuracy Rate:   97.1 %
